https://hail.is/docs/0.2/guides/basics.html#get-data-into-and-out-of-hail
    
    https://hail.is/docs/0.2/api.html

# Hail Genomic Analysis on Apache Spark

> Author:  *Tom Bresee*

>  **Why do we care ?**

*Because with research you can begin to understand how genomic variation contributes to susceptibility to human disease and to individual`s response to therapy, and thats a huge huge deal.   Effectively we will use Computational Genomics*


<br>
<br>
<br>

# Variant Dataset

"Discussions about the role of technology in genomics invariably focus on the massive growth in DNA sequencing since the beginning of the century, growth faster than Moore's law and which has led to the $1000 genome. However, future growth is projected to be even more spectacular, and to be a reality we need more powerful tools for genome analysis. Apache Spark is providing the foundation for these new tools, including two that I will cover in this talk: GATK and Hail, both open source projects from the Broad Institute. GATK and Hail are complementary: GATK provides pipelines for transforming DNA sequence data into the raw material (variant call data) needed by Hail to run genetic analysis across thousands of individuals. GATK started out originally as a single process program, but has now been ported to run on Spark at scale. Hail was written from the outset to run on Spark. In this talk I will look at how these frameworks take advantage of Spark to scale, some of the challenges in getting existing data formats to work with Spark, and some of the plans for the future."   -  Spark Sumit 2018

A typical workflow in Hail begins with importing genotype data from a standard file format such as VCF, GEN, or BGEN files into Hail’s Variant Dataset format.  

Next, samples and variants are annotated with additional meta-information such as phenotype for samples and functional consequence for variants. 

Samples, variants, and genotypes are filtered from the dataset based on expressions constructed using Hail’s Domain-Specific Language. Once the dataset has been cleaned, various analytic methods such as PCA and logistic regression are used to find genetic associations. Lastly, data is exported to a variety of file formats.  

* https://www.youtube.com/watch?v=EzdZt-5T3q0
      *  really good youtube preso on the big picture of how Hail is actually used...

Hail represents a genetic data set as a matrix where the rows are keyed by Variant objects, the columns are keyed by samples, and each cell is a Genotype object. Variant objects and Genotype objects each have methods to access attributes such as chromosome name and genotype call. Although this representation is similar to the VCF format, Hail uses a fast and storage-efficient internal representation called a Variant Dataset (VDS).

In addition to information about Samples, Variants, and Genotypes, Hail stores meta-data as annotations that can be attached to each variant (variant annotations), each sample (sample annotations), and global to the dataset (global annotations). Annotations in Hail can be thought of as a hierarchical data structure with a specific schema that is typed (similar to the JSON format).

![title](https://hail.is/docs/0.1/_images/hail-vds-rep.png)

<br>
<br>
<br>

# My Setup

* **Ubuntu on WSL**
  *  Description:    Ubuntu 18.04.2 LTS
  *  Release:        18.04
  *  Codename:       bionic
  
  
* **Apache Spark**
  *  Description:    2.4.3
  *  Hail will actually run on version 2.4.1
  
  
* **Libraries**
  *  Hail
  *  Bokeh (for advanced plotting)
  *  



<br>
<br>
<br>

# Starting the Jupyter Notebook


```

 - R E A D -
  

#  you CANT just start pyspark normally when you are using Hail 0.2 ! 
#  you have to start it with specific .jar 
#  just entering 'pyspark' will not work ! 


tbresee@TM0493322:~$ pyspark \
>   --jars $HAIL_HOME/build/libs/hail-all-spark.jar \
>   --conf spark.driver.extraClassPath=$HAIL_HOME/build/libs/hail-all-spark.jar \
>   --conf spark.executor.extraClassPath=./hail-all-spark.jar \
>   --conf spark.serializer=org.apache.spark.serializer.KryoSerializer \
>   --conf spark.kryo.registrator=is.hail.kryo.HailKryoRegistrator
[I 12:53:35.742 NotebookApp] Serving notebooks from local directory: /home/tbresee
[I 12:53:35.742 NotebookApp] The Jupyter Notebook is running at:
[I 12:53:35.743 NotebookApp] http://localhost:8888/?token=69d931e6ca06d2bc45586af80bf10a05a79b1af9bf6d6231
[I 12:53:35.743 NotebookApp]  or http://127.0.0.1:8888/?token=69d931e6ca06d2bc45586af80bf10a05a79b1af9bf6d6231
[I 12:53:35.743 NotebookApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
[W 12:53:35.750 NotebookApp] No web browser found: could not locate runnable browser.
[C 12:53:35.750 NotebookApp]

    To access the notebook, open this file in a browser:
        file:///home/tbresee/.local/share/jupyter/runtime/nbserver-10359-open.html
    Or copy and paste one of these URLs:
        http://localhost:8888/?token=69d931e6ca06d2bc45586af80bf10a05a79b1af9bf6d6231
     or http://127.0.0.1:8888/?token=69d931e6ca06d2bc45586af80bf10a05a79b1af9bf6d6231
```


* at this point, our jupyter notebook has started... 

* this is a good link to show the installation:  https://hail.is/docs/0.2/getting_started.html

In [1]:

# import the main hail library
import hail as hl


In [2]:

# import other libraries
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from math import log, isnan
import seaborn
import pyspark
import py4j


In [9]:
mt = hl.balding_nichols_model(n_populations=3, n_samples=50, n_variants=100)

Initializing Spark and Hail with default parameters...
Running on Apache Spark version 2.4.1
SparkUI available at http://10.0.75.1:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.19-c6ec8b76eb26
LOGGING: writing to /home/tbresee/hail-20190807-1257-0.2.19-c6ec8b76eb26.log
2019-08-07 17:57:44 Hail: INFO: balding_nichols_model: generating genotypes for 3 populations, 50 samples, and 100 variants...


*  Hail is *not* only a Python library; most of Hail is written in Java/Scala and runs together with Apache Spark in the Java Virtual Machine (JVM).    

*  In order to use Hail, a JVM needs to run as well. The init() function is used to initialize Hail and Spark. 

*  This function also sets global configuration parameters used for the Hail session, like the default reference genome and log file location. 

*  This function will be called automatically (with default parameters) if any Hail functionality requiring the backend (most of the libary!) is used. To initialize Hail explicitly with non-default arguments, be sure to do so directly after importing the module!


#### I am going to follow the rules here:

In [89]:
hl.citation()

'Hail Team. Hail 0.2.19-c6ec8b76eb26. https://github.com/hail-is/hail/releases/tag/0.2.19.'

<br>

## Hail

In [88]:

# please list out all hail (hl) .methods:

for the_methods in dir(hl):
    if not the_methods.startswith("_"):
        print(the_methods)

# this helps us see the big picture 


Call
GroupedMatrixTable
GroupedTable
HailType
Interval
Locus
MatrixTable
Pedigree
ReferenceGenome
Struct
Table
Trio
abs
agg
all
allele_type
any
approx_equal
argmax
argmin
array
array_scan
asc
backend
balding_nichols_model
binary_search
bind
binom_test
bit_and
bit_lshift
bit_not
bit_or
bit_rshift
bit_xor
bool
builders
call
case
ceil
chi_squared_test
citation
cite_hail
cite_hail_bibtex
coalesce
concordance
cond
context
contingency_table_test
copy_log
corr
cumulative_sum
current_backend
dbeta
de_novo
debug_info
default_reference
delimit
desc
dict
downcode
dpois
dtype
empty_array
empty_dict
empty_set
entropy
eval
eval_timed
eval_typed
exp
experimental
export_bgen
export_elasticsearch
export_gen
export_plink
export_vcf
expr
filter
filter_alleles
filter_alleles_hts
filter_intervals
find
fisher_exact_test
flatmap
flatten
float
float32
float64
floor
fold
format
fs
generic_summary
genetic_relatedness_matrix
genetics
get_reference
get_sequence
get_vcf_metadata
gp_dosage
gq_from_pl
grep
group_by


#### Rreturn the existing spark context that Hail is running on:

In [90]:
hl.spark_context()

<SparkContext master=local[*] appName=pyspark-shell>

In [91]:

# what is the default reference ? 
hl.default_reference()

# confirm: 
# Hail’s built-in references are 'GRCh37', GRCh38', and 'GRCm38'


ReferenceGenome(name=GRCh37, contigs=['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', 'X', 'Y', 'MT', 'GL000207.1', 'GL000226.1', 'GL000229.1', 'GL000231.1', 'GL000210.1', 'GL000239.1', 'GL000235.1', 'GL000201.1', 'GL000247.1', 'GL000245.1', 'GL000197.1', 'GL000203.1', 'GL000246.1', 'GL000249.1', 'GL000196.1', 'GL000248.1', 'GL000244.1', 'GL000238.1', 'GL000202.1', 'GL000234.1', 'GL000232.1', 'GL000206.1', 'GL000240.1', 'GL000236.1', 'GL000241.1', 'GL000243.1', 'GL000242.1', 'GL000230.1', 'GL000237.1', 'GL000233.1', 'GL000204.1', 'GL000198.1', 'GL000208.1', 'GL000191.1', 'GL000227.1', 'GL000228.1', 'GL000214.1', 'GL000221.1', 'GL000209.1', 'GL000218.1', 'GL000220.1', 'GL000213.1', 'GL000211.1', 'GL000199.1', 'GL000217.1', 'GL000216.1', 'GL000215.1', 'GL000205.1', 'GL000219.1', 'GL000224.1', 'GL000223.1', 'GL000195.1', 'GL000212.1', 'GL000222.1', 'GL000200.1', 'GL000193.1', 'GL000194.1', 'GL000225.1', 'GL000192.1

<br>

In [10]:
mt.count()

(100, 50)

In [14]:
# is hail a module ? 
print(type(hl))

<class 'module'>


## Initial Examination

In [15]:
print(type(mt))

<class 'hail.matrixtable.MatrixTable'>


In [83]:
# this is important !  (its a matrix table !)

In [84]:

# please list out all matrixtable.methods:

for the_methods in dir(mt):
    if not the_methods.startswith("_"):
        print(the_methods)

# thus we need a matrix table, and then you can 'do stuff'


AD
DP
GQ
GT
PL
add_col_index
add_row_index
aggregate_cols
aggregate_entries
aggregate_rows
alleles
annotate_cols
annotate_entries
annotate_globals
annotate_rows
anti_join_cols
anti_join_rows
cache
checkpoint
choose_cols
col
col_key
col_value
collect_cols_by_key
cols
compute_entry_filter_stats
count
count_cols
count_rows
describe
distinct_by_col
distinct_by_row
drop
entries
entry
explode_cols
explode_rows
filter_cols
filter_entries
filter_rows
filters
from_rows_table
globals
globals_table
group_cols_by
group_rows_by
head
index_cols
index_entries
index_globals
index_rows
info
key_cols_by
key_rows_by
localize_entries
locus
make_table
n_partitions
naive_coalesce
persist
pheno
qual
rename
repartition
row
row_key
row_value
rows
rsid
s
sample_cols
sample_qc
sample_rows
scores
select_cols
select_entries
select_globals
select_rows
semi_join_cols
semi_join_rows
show
summarize
transmute_cols
transmute_entries
transmute_globals
transmute_rows
unfilter_entries
union_cols
union_rows
unpersist
varian

In [19]:

# somple plotting libraries 
from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()


Loading BokehJS ...

<br>


**Download public 1000 Genomes data:**

We use a small chunk of the public 1000 Genomes dataset, created by downsampling the genotyped SNPs in the full VCF to about 20 MB. 

We will also integrate sample and variant metadata from separate text files.

These files are hosted by the Hail team in a public Google Storage bucket; the following cell downloads that data locally.

Note:  They keep a LOT of stuff on 'storage.googleapis.com/hail' type stuff...need to dive more into that 


In [20]:
hl.utils.get_1kg('data/')

2019-08-07 18:32:52 Hail: INFO: downloading 1KG VCF ...
  Source: https://storage.googleapis.com/hail-tutorial/1kg.vcf.bgz
2019-08-07 18:32:56 Hail: INFO: importing VCF and writing to matrix table...
2019-08-07 18:32:58 Hail: INFO: Coerced sorted dataset
2019-08-07 18:33:03 Hail: INFO: wrote matrix table with 10961 rows and 284 columns in 16 partitions to data/1kg.mt
2019-08-07 18:33:03 Hail: INFO: downloading 1KG annotations ...
  Source: https://storage.googleapis.com/hail-tutorial/1kg_annotations.txt
2019-08-07 18:33:03 Hail: INFO: downloading Ensembl gene annotations ...
  Source: https://storage.googleapis.com/hail-tutorial/ensembl_gene_annotations.txt
2019-08-07 18:33:04 Hail: INFO: Done!


In [97]:

# Next we read the written file, assigning the variable mt (for matrix table).

mt = hl.read_matrix_table('data/1kg.mt')

# this is a matrix table 



**Getting to know our data:**
    
    
It’s important to have easy ways to slice, dice, query, and summarize a dataset. Some of this functionality is demonstrated below.

The `rows` method can be used to get a table with all the row fields in our MatrixTable.

We can use `rows` along with `select` to pull out 5 variants. The `select` method takes either a string refering to a field name in the table, or a Hail Expression. Here, we leave the arguments blank to keep only the row key fields, locus and alleles.

Use the show method (which spark uses as well) to display the variants.


In [22]:

mt.rows().select().show(5)


locus,alleles
locus<GRCh37>,array<str>
1:904165,"[""G"",""A""]"
1:909917,"[""G"",""A""]"
1:986963,"[""C"",""T""]"
1:1563691,"[""T"",""G""]"
1:1707740,"[""T"",""G""]"


In [23]:

mt.rows().select().show(100)


locus,alleles
locus<GRCh37>,array<str>
1:904165,"[""G"",""A""]"
1:909917,"[""G"",""A""]"
1:986963,"[""C"",""T""]"
1:1563691,"[""T"",""G""]"
1:1707740,"[""T"",""G""]"
1:2252970,"[""C"",""T""]"
1:2284195,"[""T"",""C""]"
1:2779043,"[""T"",""C""]"
1:2944527,"[""G"",""A""]"


In [25]:
mt.row_key.show(5)

locus,alleles
locus<GRCh37>,array<str>
1:904165,"[""G"",""A""]"
1:909917,"[""G"",""A""]"
1:986963,"[""C"",""T""]"
1:1563691,"[""T"",""G""]"
1:1707740,"[""T"",""G""]"


In [26]:

# Here is how to peek at the first few sample IDs:
mt.s.show(5)


s
str
"""HG00096"""
"""HG00099"""
"""HG00105"""
"""HG00118"""
"""HG00129"""



To look at the first few genotype calls, we can use entries along with `select` and `take`. The `take` method collects the first n rows into a list. Alternatively, we can use the `show` method, which prints the first n rows to the console in a table format.

Try changing `take` to `show` in the cell below.


In [27]:
mt.entry.take(5)

[Struct(GT=Call(alleles=[0, 0], phased=False), AD=[4, 0], DP=4, GQ=12, PL=[0, 12, 147]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[8, 0], DP=8, GQ=24, PL=[0, 24, 315]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[8, 0], DP=8, GQ=23, PL=[0, 23, 230]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[7, 0], DP=7, GQ=21, PL=[0, 21, 270]),
 Struct(GT=Call(alleles=[0, 0], phased=False), AD=[5, 0], DP=5, GQ=15, PL=[0, 15, 205])]

**Adding column fields**


A Hail MatrixTable can have any number of row fields and column fields for storing data associated with each row and column. Annotations are usually a critical part of any genetic study. Column fields are where you’ll store information about sample phenotypes, ancestry, sex, and covariates. Row fields can be used to store information like gene membership and functional impact for use in QC or analysis.

In this tutorial, we demonstrate how to take a text file and use it to annotate the columns in a MatrixTable.

The file provided contains the sample ID, the population and “super-population” designations, the sample sex, and two simulated phenotypes (one binary, one discrete).

This file can be imported into Hail with import_table. This function produces a Table object. Think of this as a Pandas or R dataframe that isn’t limited by the memory on your machine – behind the scenes, it’s distributed with Spark.

In [28]:

table = (hl.import_table('data/1kg_annotations.txt', impute=True)
         .key_by('Sample'))


2019-08-07 18:43:01 Hail: INFO: Reading table to impute column types
2019-08-07 18:43:01 Hail: INFO: Finished type imputation
  Loading column 'Sample' as type 'str' (imputed)
  Loading column 'Population' as type 'str' (imputed)
  Loading column 'SuperPopulation' as type 'str' (imputed)
  Loading column 'isFemale' as type 'bool' (imputed)
  Loading column 'PurpleHair' as type 'bool' (imputed)
  Loading column 'CaffeineConsumption' as type 'int32' (imputed)


In [29]:

# A good way to peek at the structure of a Table is to look at its schema.
table.describe()


----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'Sample': str 
    'Population': str 
    'SuperPopulation': str 
    'isFemale': bool 
    'PurpleHair': bool 
    'CaffeineConsumption': int32 
----------------------------------------
Key: ['Sample']
----------------------------------------


In [30]:

# To peek at the first few values, use the show method:

table.show(width=100)

# you can use show with standard apache spark stuff as well 


Sample,Population,SuperPopulation,isFemale,PurpleHair,CaffeineConsumption
str,str,str,bool,bool,int32
"""HG00096""","""GBR""","""EUR""",false,false,4
"""HG00097""","""GBR""","""EUR""",true,true,4
"""HG00098""","""GBR""","""EUR""",false,false,5
"""HG00099""","""GBR""","""EUR""",true,false,4
"""HG00100""","""GBR""","""EUR""",true,false,5
"""HG00101""","""GBR""","""EUR""",false,true,1
"""HG00102""","""GBR""","""EUR""",true,true,6
"""HG00103""","""GBR""","""EUR""",false,true,5
"""HG00104""","""GBR""","""EUR""",true,false,5



Now we’ll use this table to add sample annotations to our dataset, storing the annotations in column fields in our MatrixTable. First, we’ll print the existing column schema:
    


In [31]:
print(mt.col.dtype)

struct{s: str}


We use the annotate_cols method to join the table with the MatrixTable containing our dataset.

In [32]:
mt = mt.annotate_cols(pheno = table[mt.s])

In [33]:
mt.col.describe()

--------------------------------------------------------
Type:
        struct {
        s: str, 
        pheno: struct {
            Population: str, 
            SuperPopulation: str, 
            isFemale: bool, 
            PurpleHair: bool, 
            CaffeineConsumption: int32
        }
    }
--------------------------------------------------------
Source:
Index:
    ['column']
--------------------------------------------------------



**Query functions and the Hail Expression Language**

Hail has a number of useful query functions that can be used for gathering statistics on our dataset. These query functions take Hail Expressions as arguments.

We will start by looking at some statistics of the information in our table. The aggregate method can be used to aggregate over rows of the table.

counter is an aggregation function that counts the number of occurrences of each unique element. We can use this to pull out the population distribution by passing in a Hail Expression for the field that we want to count by.


In [35]:
pprint(table.aggregate(hl.agg.counter(table.SuperPopulation)))

{'AFR': 1018, 'AMR': 535, 'EAS': 617, 'EUR': 669, 'SAS': 661}


2019-08-07 18:45:30 Hail: INFO: Coerced sorted dataset


`stats` is an aggregation function that produces some useful statistics about numeric collections. We can use this to see the distribution of the CaffeineConsumption phenotype.

In [36]:
pprint(table.aggregate(hl.agg.stats(table.CaffeineConsumption)))

{'max': 10.0,
 'mean': 3.9837142857142855,
 'min': -1.0,
 'n': 3500,
 'stdev': 1.7021055628070711,
 'sum': 13943.0}


2019-08-07 18:45:57 Hail: INFO: Coerced sorted dataset


However, these metrics aren’t perfectly representative of the samples in our dataset. Here’s why:

In [37]:
table.count()

3500

In [38]:
mt.count_cols()

284


Since there are fewer samples in our dataset than in the full thousand genomes cohort, we need to look at annotations on the dataset. We can use `aggregate_cols` to get the metrics for only the samples in our dataset.


In [39]:
mt.aggregate_cols(hl.agg.counter(mt.pheno.SuperPopulation))

2019-08-07 18:47:13 Hail: INFO: Coerced sorted dataset


{'AFR': 76, 'EAS': 72, 'AMR': 34, 'SAS': 55, 'EUR': 47}

In [40]:
pprint(mt.aggregate_cols(hl.agg.stats(mt.pheno.CaffeineConsumption)))

{'max': 9.0,
 'mean': 4.415492957746479,
 'min': 0.0,
 'n': 284,
 'stdev': 1.577763427465917,
 'sum': 1254.0}


2019-08-07 18:47:18 Hail: INFO: Coerced sorted dataset



The functionality demonstrated in the last few cells isn’t anything especially new: it’s certainly not difficult to ask these questions with Pandas or R dataframes, or even Unix tools like awk. But Hail can use the same interfaces and query language to analyze collections that are much larger, like the set of variants.

Here we calculate the counts of each of the 12 possible unique SNPs (4 choices for the reference base * 3 choices for the alternate base).

To do this, we need to get the alternate allele of each variant and then count the occurences of each unique ref/alt pair. This can be done with Hail’s counter function.


In [41]:
snp_counts = mt.aggregate_rows(hl.agg.counter(hl.Struct(ref=mt.alleles[0], alt=mt.alleles[1])))
pprint(snp_counts)

{Struct(ref='T', alt='A'): 79,
 Struct(ref='T', alt='C'): 1879,
 Struct(ref='G', alt='T'): 480,
 Struct(ref='A', alt='G'): 1944,
 Struct(ref='T', alt='G'): 468,
 Struct(ref='G', alt='C'): 112,
 Struct(ref='G', alt='A'): 2387,
 Struct(ref='A', alt='T'): 76,
 Struct(ref='C', alt='T'): 2436,
 Struct(ref='C', alt='G'): 150,
 Struct(ref='C', alt='A'): 496,
 Struct(ref='A', alt='C'): 454}


We can list the counts in descending order using Python’s Counter class.



In [42]:
from collections import Counter
counts = Counter(snp_counts)
counts.most_common()

[(Struct(ref='C', alt='T'), 2436),
 (Struct(ref='G', alt='A'), 2387),
 (Struct(ref='A', alt='G'), 1944),
 (Struct(ref='T', alt='C'), 1879),
 (Struct(ref='C', alt='A'), 496),
 (Struct(ref='G', alt='T'), 480),
 (Struct(ref='T', alt='G'), 468),
 (Struct(ref='A', alt='C'), 454),
 (Struct(ref='C', alt='G'), 150),
 (Struct(ref='G', alt='C'), 112),
 (Struct(ref='T', alt='A'), 79),
 (Struct(ref='A', alt='T'), 76)]



It’s nice to see that we can actually uncover something biological from this small dataset: we see that these frequencies come in pairs. C/T and G/A are actually the same mutation, just viewed from from opposite strands. Likewise, T/A and A/T are the same mutation on opposite strands. There’s a 30x difference between the frequency of C/T and A/T SNPs. Why?

The same Python, R, and Unix tools could do this work as well, but we’re starting to hit a wall - the latest gnomAD release (http://gnomad.broadinstitute.org/) publishes about 250 million variants, and that won’t fit in memory on a single computer.

What about genotypes? Hail can query the collection of all genotypes in the dataset, and this is getting large even for our tiny dataset. Our 284 samples and 10,000 variants produce 10 million unique genotypes. The gnomAD dataset has about 5 trillion unique genotypes.

Hail plotting functions allow Hail fields as arguments, so we can pass in the DP field directly here. If the range and bins arguments are not set, this function will compute the range based on minimum and maximum values of the field and use the default 50 bins.




In [43]:
p = hl.plot.histogram(mt.DP, range=(0,30), bins=30, title='DP Histogram', legend='DP')
show(p)


**Quality Control**

QC is where analysts spend most of their time with sequencing datasets. QC is an iterative process, and is different for every project: there is no “push-button” solution for QC. Each time the Broad collects a new group of samples, it finds new batch effects. However, by practicing open science and discussing the QC process and decisions with others, we can establish a set of best practices as a community.

QC is entirely based on the ability to understand the properties of a dataset. Hail attempts to make this easier by providing the sample_qc function, which produces a set of useful metrics and stores them in a column field.


In [44]:
mt.col.describe()

--------------------------------------------------------
Type:
        struct {
        s: str, 
        pheno: struct {
            Population: str, 
            SuperPopulation: str, 
            isFemale: bool, 
            PurpleHair: bool, 
            CaffeineConsumption: int32
        }
    }
--------------------------------------------------------
Source:
Index:
    ['column']
--------------------------------------------------------


In [45]:
mt = hl.sample_qc(mt)

In [46]:
mt.col.describe()

--------------------------------------------------------
Type:
        struct {
        s: str, 
        pheno: struct {
            Population: str, 
            SuperPopulation: str, 
            isFemale: bool, 
            PurpleHair: bool, 
            CaffeineConsumption: int32
        }, 
        sample_qc: struct {
            dp_stats: struct {
                mean: float64, 
                stdev: float64, 
                min: float64, 
                max: float64
            }, 
            gq_stats: struct {
                mean: float64, 
                stdev: float64, 
                min: float64, 
                max: float64
            }, 
            call_rate: float64, 
            n_called: int64, 
            n_not_called: int64, 
            n_filtered: int64, 
            n_hom_ref: int64, 
            n_het: int64, 
            n_hom_var: int64, 
            n_non_ref: int64, 
            n_singleton: int64, 
            n_snp: int64, 
            n_insertio

Plotting the QC metrics is a good place to start.


In [47]:
p = hl.plot.histogram(mt.sample_qc.call_rate, range=(.88,1), legend='Call Rate')
show(p)

2019-08-07 18:50:31 Hail: INFO: Coerced sorted dataset


In [48]:
p = hl.plot.histogram(mt.sample_qc.gq_stats.mean, range=(10,70), legend='Mean Sample GQ')
show(p)

2019-08-07 18:50:43 Hail: INFO: Coerced sorted dataset


Often, these metrics are correlated.

In [50]:
p = hl.plot.scatter(mt.sample_qc.dp_stats.mean, mt.sample_qc.call_rate, xlabel='Mean DP', ylabel='Call Rate')
show(p)

2019-08-07 18:51:03 Hail: INFO: Coerced sorted dataset



Removing outliers from the dataset will generally improve association results. We can make arbitrary cutoffs and use them to filter:
    

In [51]:
mt = mt.filter_cols((mt.sample_qc.dp_stats.mean >= 4) & (mt.sample_qc.call_rate >= 0.97))
print('After filter, %d/284 samples remain.' % mt.count_cols())

After filter, 250/284 samples remain.


Next is genotype QC. It’s a good idea to filter out genotypes where the reads aren’t where they should be: if we find a genotype called homozygous reference with >10% alternate reads, a genotype called homozygous alternate with >10% reference reads, or a genotype called heterozygote without a ref / alt balance near 1:1, it is likely to be an error.

In a low-depth dataset like 1KG, it is hard to detect bad genotypes using this metric, since a read ratio of 1 alt to 10 reference can easily be explained by binomial sampling. However, in a high-depth dataset, a read ratio of 10:100 is a sure cause for concern!

In [52]:
ab = mt.AD[1] / hl.sum(mt.AD)

filter_condition_ab = ((mt.GT.is_hom_ref() & (ab <= 0.1)) |
                        (mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (mt.GT.is_hom_var() & (ab >= 0.9)))

fraction_filtered = mt.aggregate_entries(hl.agg.fraction(~filter_condition_ab))
print(f'Filtering {fraction_filtered * 100:.2f}% entries out of downstream analysis.')
mt = mt.filter_entries(filter_condition_ab)

Filtering 3.64% entries out of downstream analysis.


Variant QC is a bit more of the same: we can use the variant_qc function to produce a variety of useful statistics, plot them, and filter.

In [53]:
mt = hl.variant_qc(mt)

In [54]:
mt.row.describe()

--------------------------------------------------------
Type:
        struct {
        locus: locus<GRCh37>, 
        alleles: array<str>, 
        rsid: str, 
        qual: float64, 
        filters: set<str>, 
        info: struct {
            AC: array<int32>, 
            AF: array<float64>, 
            AN: int32, 
            BaseQRankSum: float64, 
            ClippingRankSum: float64, 
            DP: int32, 
            DS: bool, 
            FS: float64, 
            HaplotypeScore: float64, 
            InbreedingCoeff: float64, 
            MLEAC: array<int32>, 
            MLEAF: array<float64>, 
            MQ: float64, 
            MQ0: int32, 
            MQRankSum: float64, 
            QD: float64, 
            ReadPosRankSum: float64, 
            set: str
        }, 
        variant_qc: struct {
            dp_stats: struct {
                mean: float64, 
                stdev: float64, 
                min: float64, 
                max: float64
            }, 

These statistics actually look pretty good: we don’t need to filter this dataset. Most datasets require thoughtful quality control, though. The filter_rows method can help!

**Let’s do a GWAS!**

First, we need to restrict to variants that are :

common (we’ll use a cutoff of 1%)

not so far from Hardy-Weinberg (https://en.wikipedia.org/wiki/Hardy%E2%80%93Weinberg_principle) equilibrium as to suggest sequencing error


In [55]:
mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.01)

In [56]:
mt = mt.filter_rows(mt.variant_qc.p_value_hwe > 1e-6)

In [57]:
print('Samples: %d  Variants: %d' % (mt.count_cols(), mt.count_rows()))

Samples: 250  Variants: 7849


These filters removed about 15% of sites (we started with a bit over 10,000). This is NOT representative of most sequencing datasets! We have already downsampled the full thousand genomes dataset to include more common variants than we’d expect by chance.

In Hail, the association tests accept column fields for the sample phenotype and covariates. Since we’ve already got our phenotype of interest (caffeine consumption) in the dataset, we are good to go:

In [58]:
gwas = hl.linear_regression_rows(y=mt.pheno.CaffeineConsumption,
                                 x=mt.GT.n_alt_alleles(),
                                 covariates=[1.0])
gwas.row.describe()

2019-08-07 18:53:39 Hail: INFO: linear_regression_rows: running on 250 samples for 1 response variable y,
    with input variable x, and 1 additional covariate...


--------------------------------------------------------
Type:
        struct {
        locus: locus<GRCh37>, 
        alleles: array<str>, 
        n: int32, 
        sum_x: float64, 
        y_transpose_x: float64, 
        beta: float64, 
        standard_error: float64, 
        t_stat: float64, 
        p_value: float64
    }
--------------------------------------------------------
Source:
Index:
    ['row']
--------------------------------------------------------



Looking at the bottom of the above printout, you can see the linear regression adds new row fields for the beta, standard error, t-statistic, and p-value.

Hail makes it easy to visualize results! Let’s make a Manhattan (https://en.wikipedia.org/wiki/Manhattan_plot) plot:
    

In [60]:
p = hl.plot.manhattan(gwas.p_value)
show(p)


This doesn’t look like much of a skyline. 

Let’s check whether our GWAS was well controlled using a Q-Q (quantile-quantile) plot.

https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot


In [62]:
p = hl.plot.qq(gwas.p_value)
show(p)

2019-08-07 18:54:47 Hail: INFO: Ordering unsorted dataset with network shuffle



**???**

The observed p-values drift away from the expectation immediately. Either every SNP in our dataset is causally linked to caffeine consumption (unlikely), or there’s a confounder.

We didn’t tell you, but sample ancestry was actually used to simulate this phenotype. This leads to a stratified (https://en.wikipedia.org/wiki/Population_stratification) distribution of the phenotype. The solution is to include ancestry as a covariate in our regression.

The linear_regression_rows (`https://hail.is/docs/0.2/methods/stats.html`) function can also take column fields to use as covariates. We already annotated our samples with reported ancestry, but it is good to be skeptical of these labels due to human error. Genomes don’t have that problem! Instead of using reported ancestry, we will use genetic ancestry by including computed principal components in our model.

The pca (https://hail.is/docs/0.2/methods/stats.html) function produces eigenvalues as a list and sample PCs as a Table, and can also produce variant loadings when asked. The hwe_normalized_pca function does the same, using HWE-normalized genotypes for the PCA.




In [63]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt.GT)

2019-08-07 18:57:07 Hail: INFO: hwe_normalized_pca: running PCA using 7841 variants.
2019-08-07 18:57:10 Hail: INFO: pca: running PCA with 10 components...


In [64]:
pprint(eigenvalues)

[18.02377947184685,
 9.988945550363265,
 3.5383122629171258,
 2.657759078373002,
 1.5966032147658389,
 1.5416611649602379,
 1.502987224878172,
 1.4720816378531145,
 1.4678188487330752,
 1.447783520133496]


In [65]:
pcs.show(5, width=100)

s,scores
str,array<float64>
"""HG00096""","[-1.22e-01,-2.81e-01,1.11e-01,-1.28e-01,6.81e-02,-3.72e-03,-2.66e-02,4.99e-03,-9.33e-02,-1.48..."
"""HG00099""","[-1.13e-01,-2.90e-01,1.08e-01,-7.04e-02,4.20e-02,3.33e-02,1.61e-02,-1.15e-03,3.29e-02,2.33e-02]"
"""HG00105""","[-1.08e-01,-2.80e-01,1.03e-01,-1.05e-01,9.40e-02,1.27e-02,3.14e-02,3.08e-02,1.06e-02,-1.93e-02]"
"""HG00118""","[-1.25e-01,-2.98e-01,7.21e-02,-1.07e-01,2.89e-02,8.09e-03,-4.70e-02,-3.32e-02,-2.59e-04,8.49e..."
"""HG00129""","[-1.07e-01,-2.87e-01,9.72e-02,-1.16e-01,1.38e-02,1.87e-02,-8.37e-02,-4.87e-02,3.73e-02,2.11e-02]"


Now that we’ve got principal components per sample, we may as well plot them! Human history exerts a strong effect in genetic datasets. Even with a 50MB sequencing dataset, we can recover the major human populations.

In [66]:
mt = mt.annotate_cols(scores = pcs[mt.s].scores)

In [67]:
p = hl.plot.scatter(mt.scores[0],
                    mt.scores[1],
                    label=mt.pheno.SuperPopulation,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)

2019-08-07 18:58:01 Hail: INFO: Coerced sorted dataset


Now we can rerun our linear regression, controlling for sample sex and the first few principal components. We’ll do this with input variable the number of alternate alleles as before, and again with input variable the genotype dosage derived from the PL field.

In [68]:
gwas = hl.linear_regression_rows(
    y=mt.pheno.CaffeineConsumption,
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.pheno.isFemale, mt.scores[0], mt.scores[1], mt.scores[2]])

2019-08-07 18:58:18 Hail: INFO: linear_regression_rows: running on 250 samples for 1 response variable y,
    with input variable x, and 5 additional covariates...


We’ll first make a Q-Q plot to assess inflation…



In [69]:
p = hl.plot.qq(gwas.p_value)
show(p)

2019-08-07 18:58:40 Hail: INFO: Ordering unsorted dataset with network shuffle


That’s more like it! This shape is indicative of a well-controlled (but not especially well-powered) study. And now for the Manhattan plot:

In [70]:
p = hl.plot.manhattan(gwas.p_value)
show(p)

**Rare variant analysis**

Here we’ll demonstrate how one can use the expression language to group and count by any arbitrary properties in row and column fields. Hail also implements the sequence kernel association test (SKAT).


In [72]:

entries = mt.entries()
results = (entries.group_by(pop = entries.pheno.SuperPopulation, chromosome = entries.locus.contig)
      .aggregate(n_het = hl.agg.count_where(entries.GT.is_het())))


2019-08-07 18:59:36 Hail: WARN: entries(): Resulting entries table is sorted by '(row_key, col_key)'.
    To preserve row-major matrix table order, first unkey columns with 'key_cols_by()'


In [73]:

results.show()


2019-08-07 18:59:58 Hail: INFO: Ordering unsorted dataset with network shuffle


pop,chromosome,n_het
str,str,int64
"""AFR""","""1""",11276
"""AFR""","""10""",7160
"""AFR""","""11""",6875
"""AFR""","""12""",7048
"""AFR""","""13""",4678
"""AFR""","""14""",4313
"""AFR""","""15""",3904
"""AFR""","""16""",4593
"""AFR""","""17""",3718


What if we want to group by minor allele frequency bin and hair color, and calculate the mean GQ?

In [75]:

entries = entries.annotate(maf_bin = hl.cond(entries.info.AF[0]<0.01, "< 1%",
                             hl.cond(entries.info.AF[0]<0.05, "1%-5%", ">5%")))

results2 = (entries.group_by(af_bin = entries.maf_bin, purple_hair = entries.pheno.PurpleHair)
      .aggregate(mean_gq = hl.agg.stats(entries.GQ).mean,
                 mean_dp = hl.agg.stats(entries.DP).mean))


In [76]:
results2.show()

2019-08-07 19:00:38 Hail: INFO: Coerced sorted dataset
2019-08-07 19:00:38 Hail: INFO: Coerced dataset with out-of-order partitions.


af_bin,purple_hair,mean_gq,mean_dp
str,bool,float64,float64
"""1%-5%""",false,2.48e+01,7.43e+00
"""1%-5%""",true,2.46e+01,7.47e+00
"""< 1%""",false,2.35e+01,7.55e+00
"""< 1%""",true,2.35e+01,7.53e+00
""">5%""",false,3.70e+01,7.65e+00
""">5%""",true,3.73e+01,7.70e+00



We’ve shown that it’s easy to aggregate by a couple of arbitrary statistics. This specific examples may not provide especially useful pieces of information, but this same pattern can be used to detect effects of rare variation:

* Count the number of heterozygous genotypes per gene by functional category (synonymous, missense, or loss-of-function) to estimate per-gene functional constraint

* Count the number of singleton loss-of-function mutations per gene in cases and controls to detect genes involved in disease


### Mini-Ref


```
For reference, here’s the full workflow to all tutorial endpoints combined into one cell.

table = hl.import_table('data/1kg_annotations.txt', impute=True).key_by('Sample')

mt = hl.read_matrix_table('data/1kg.mt')
mt = mt.annotate_cols(pheno = table[mt.s])
mt = hl.sample_qc(mt)
mt = mt.filter_cols((mt.sample_qc.dp_stats.mean >= 4) & (mt.sample_qc.call_rate >= 0.97))
ab = mt.AD[1] / hl.sum(mt.AD)
filter_condition_ab = ((mt.GT.is_hom_ref() & (ab <= 0.1)) |
                        (mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (mt.GT.is_hom_var() & (ab >= 0.9)))
mt = mt.filter_entries(filter_condition_ab)
mt = hl.variant_qc(mt)
mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.01)

eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt.GT)

mt = mt.annotate_cols(scores = pcs[mt.s].scores)
gwas = hl.linear_regression_rows(
    y=mt.pheno.CaffeineConsumption,
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.pheno.isFemale, mt.scores[0], mt.scores[1], mt.scores[2]])

```


<br>
<br>
<br>
<br>


# Lets examine the basics now after that example

In [99]:
!pip install pyvcf

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
  Stored in directory: /home/tbresee/.cache/pip/wheels/81/91/41/3272543c0b9c61da9c525f24ee35bae6fe8f60d4858c66805d
Successfully built pyvcf
Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/usr/local/lib/python2.7/dist-packages/PyVCF-0.6.8.dist-info'
Consider using the `--user` option or check the permissions.

You are using pip version 19.0.3, however version 19.2.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [100]:
!pip3 install pyvcf

  Cache entry deserialization failed, entry ignored
  Cache entry deserialization failed, entry ignored
  Using cached https://files.pythonhosted.org/packages/ec/51/f45cea425fd5cb0b0380f5b0f048ebc1da5b417e48d304838c02d6288a1e/setuptools-41.0.1-py2.py3-none-any.whl
  Running setup.py bdist_wheel for pyvcf ... done
  Stored in directory: /home/tbresee/.cache/pip/wheels/81/91/41/3272543c0b9c61da9c525f24ee35bae6fe8f60d4858c66805d
Successfully built pyvcf


In [109]:
import vcf

FileNotFoundError: [Errno 2] No such file or directory: 'E:/scaffold612.vcf'